In [2]:
from together import Together
import pandas as pd
import numpy as np
from datasets import load_dataset
from tokens import tokens
import os
import dspy
import textgrad
from textgrad.engine import get_engine
from textgrad import Variable
from textgrad.optimizer import TextualGradientDescent
from textgrad.loss import TextLoss
from textgrad.tasks import DataLoader 
import concurrent
from tqdm import tqdm
import random

c:\Users\mikad\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
c:\Users\mikad\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


## Initialization

In [3]:
# set up tokens

os.environ['TOGETHER_API_KEY'] = together_token
os.environ['HF_TOKEN'] = hf_token

In [4]:
# Load testing data
dataset = load_dataset("gtfintechlab/fomc_communication")
sample = dataset['test'][0]
sentence = sample['sentence']
label = sample['label']
print(f"Original sentence: {sentence}\nLabel: {label}")

Original sentence: Participants agreed that the labor market had remained strong over the intermeeting period and that economic activity had risen at a moderate rate.
Label: 2


c:\Users\mikad\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
# the one in prompts.py was not returning just the system prompt/user message which together 1.2 needs
def fomc_prompt(sentence: str):

    system_prompt = f"""Discard all the previous instructions. Behave like you are an expert sentence clas-
                sifier."""
    user_msg = f"""Classify the following sentence from FOMC into ‘HAWKISH’, ‘DOVISH’, or ‘NEU-
                TRAL’ class. Label ‘HAWKISH’ if it is corresponding to tightening of the monetary policy,
                ‘DOVISH’ if it is corresponding to easing of the monetary policy, or ‘NEUTRAL’ if the
                stance is neutral. Provide the label in the first line and provide a short explanation in the
                second line. This is the sentence: {sentence}"""

    return system_prompt, user_msg

## Initial textgrad testing

In [7]:
# basic textgrad together engine
engine = get_engine('together-allenai/OLMo-7B-Instruct')
engine(fomc_prompt(sentence)[1], system_prompt = fomc_prompt(sentence)[0], max_tokens=128, temperature=0.7, top_p=0.7)

'Neutral class\nExplanation: The sentence indicates that participants agreed on the strong labor market and moderate economic activity, suggesting a neutral stance towards monetary policy.'

In [9]:
system_prompt = 'Discard all the previous instructions. Behave like you are an expert sentence classifier.'
engine = get_engine('together-mistralai/Mistral-7B-Instruct-v0.3')

sys_prompt = Variable('Discard all the previous instructions. Behave like you are an expert sentence classifier.', role_description="The system prompt")
user_prompt = Variable("Classify the sentence's stance on the monetary policy between hawkish, neutral, and dovish.", role_description="The user prompt", requires_grad=True)
input_sentence = Variable(sentence, role_description="The input sentence")
loss = TextLoss(sys_prompt, engine=engine)

# optimization does not allow you to set any parameters besides prompt & system prompt
# meaning you can't set max tokens or temperature --> problem b/c default output tokens is 2000 
# so models with too small context windows will error

optimizer = TextualGradientDescent(parameters=[user_prompt], engine=engine)
l = loss(input_sentence)
l.backward(engine)
optimizer.step()

In [10]:
# optimized version of user prompt
user_prompt.value

'Determine the monetary policy stance (hawkish, neutral, or dovish) expressed in the given sentence.'

## Textgrad Prompt Optimization

In [6]:
def find_first_occurrence(text):
    words = ["hawkish", "dovish", "neutral"]
    positions = [(text.find(word), word) for word in words if text.find(word) != -1]
    return min(positions)[1] if positions else None

def string_based_equality_fn(prediction: Variable, ground_truth_answer: Variable):
    pred = find_first_occurrence(str(prediction.value).lower())
    return pred != None and str(pred).lower() == str(ground_truth_answer.value.lower()).lower()

eval_fn = textgrad.autograd.StringBasedFunction(string_based_equality_fn, "Evaluate if LLM answer matches ground truth")

In [7]:
def eval_sample(item, eval_fn, model):
    x, y = item
    x = Variable(x, requires_grad=False, role_description="query to the language model")
    y = Variable(y, requires_grad=False, role_description="correct answer for the query")
    response = model(x)
    eval_output_variable = eval_fn(inputs=dict(prediction=response, ground_truth_answer=y))
    return int(eval_output_variable.value == 'True')
    
def eval_dataset(test_set, eval_fn, model, max_samples = None):
    if max_samples is None: 
        max_samples = len(test_set)
    accuracy_list = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=16) as executor:
        futures = []
        for _, sample in enumerate(test_set):
            future = executor.submit(eval_sample, sample, eval_fn, model)
            futures.append(future)
            if len(futures) >= max_samples:
                break
        tqdm_loader = tqdm(concurrent.futures.as_completed(futures), total=len(futures), position=0)
        for future in tqdm_loader:
            acc_item = future.result()
            accuracy_list.append(acc_item)
            tqdm_loader.set_description(f"Accuracy: {np.mean(accuracy_list)}")
    return accuracy_list 

def run_validation_revert(system_prompt, results, eval_fn, model, val_set):
    val_accuracy = np.mean(eval_dataset(val_set, eval_fn, model))
    prev_accuracy = np.mean(results["val_accuracy"][-1])
    print(f"Validation accuracy: {val_accuracy}\nPrevious validation accuracy: {prev_accuracy}")
    previous_prompt = results["prompt"][-1]
    
    if val_accuracy < prev_accuracy:
        print(f"Rejected prompt: {system_prompt.value}")
        system_prompt.set_value(previous_prompt)
        val_accuracy = prev_accuracy

    results["val_accuracy"].append(val_accuracy)

In [8]:
engine = get_engine('together-mistralai/Mistral-7B-Instruct-v0.3')

starting_prompt = "Classify the following sentence from FOMC into hawkish, dovish, or neutral based on its stance on monetary policy."
system_prompt = Variable(starting_prompt, requires_grad=True, role_description="system prompt to the language model")
model = textgrad.BlackboxLLM(engine, system_prompt)

optimizer = TextualGradientDescent(engine=engine, parameters=[system_prompt])

In [69]:
# testing zero-shot performance
mapping = {0: 'dovish', 1: 'hawkish', 2: 'neutral'}
data = [(data['sentence'], mapping[data['label']]) for data in dataset['test']]
results = {"test_accuracy": [], "prompt": []}
results["test_accuracy"].append(eval_dataset(data, eval_fn, model))
results["prompt"].append(system_prompt.get_value())

Accuracy: 0.6048387096774194: 100%|██████████| 496/496 [00:00<00:00, 625.43it/s]


In [19]:
# subset the data to do a mini run
mapping = {0: 'dovish', 1: 'hawkish', 2: 'neutral'}
def pick_random(data, n = 20):
    labels = {'dovish': [], 'hawkish': [], 'neutral': []}
    for sentence, label in data:
        labels[label].append((sentence, label))
    result = []
    for label in labels:
        result.extend(random.sample(labels[label], n))
    return result
training_data = [(data['sentence'], mapping[data['label']]) for data in dataset['train']]
testing_data = [(data['sentence'], mapping[data['label']]) for data in dataset['test']]
random.shuffle(testing_data)
val_data_options = testing_data[:100]
testing_data = testing_data[100:]
training_data = pick_random(training_data, n = 30)
val_data = pick_random(val_data_options, n = 12)
testing_data = pick_random(testing_data, n = 12)

In [24]:
results = {"test_accuracy": [0], "prompt": [starting_prompt], "val_accuracy": [0]}
train_loader = DataLoader(training_data, batch_size=30, shuffle=True)
num_epochs = 5
for epoch in range(num_epochs):
    for steps, (batch_x, batch_y) in enumerate((pbar := tqdm(train_loader, position=0))):
        pbar.set_description(f"Training step {steps}. Epoch {epoch}")
        optimizer.zero_grad()
        losses = []
        for (x, y) in zip(batch_x, batch_y):
            x = Variable(x, requires_grad=False, role_description="query to the language model")
            y = Variable(y, requires_grad=False, role_description="correct answer for the query")
            response = model(x)
            eval_output_variable = eval_fn(inputs=dict(prediction=response, ground_truth_answer=y))
            losses.append(eval_output_variable)
        total_loss = textgrad.sum(losses)
        total_loss.backward(engine)
        optimizer.step()
        
        run_validation_revert(system_prompt, results, eval_fn, model, val_data)
        
        print("sys prompt: ", system_prompt)
        test_acc = eval_dataset(testing_data, eval_fn, model)
        results["test_accuracy"].append(test_acc)
        results["prompt"].append(system_prompt.get_value())
        
        if steps >= 3:
            break

Accuracy: 0.4722222222222222: 100%|██████████| 36/36 [00:46<00:00,  1.29s/it] 


Validation accuracy: 0.4722222222222222
Previous validation accuracy: 0.0
sys prompt:  Classify the given sentence from FOMC as either hawkish, dovish, or neutral. A hawkish stance suggests a more aggressive approach to monetary policy, while a dovish stance indicates a more accommodative approach. A neutral stance implies no significant change in the monetary policy stance. Provide a clear and concise explanation, highlighting specific phrases or arguments that support your classification. If the sentence is neutral, please explain why it does not express a clear bias towards tightening or easing monetary policy.


Accuracy: 0.4444444444444444: 100%|██████████| 36/36 [00:49<00:00,  1.37s/it] 


Validation accuracy: 0.4444444444444444
Previous validation accuracy: 0.4722222222222222
Rejected prompt: For the purpose of this task, 'hawkish' refers to a central bank's policy stance that is more likely to raise interest rates to control inflation, while 'dovish' refers to a policy stance that is more likely to keep interest rates low to stimulate economic growth. Analyze the given sentence from FOMC and classify it as either hawkish (leaning towards a more aggressive approach to tightening monetary policy), neutral (neither hawkish nor dovish), or dovish (leaning towards a more accommodative approach to easing monetary policy). Provide a clear and concise explanation, highlighting specific phrases or arguments that support your classification. If the sentence is neutral, please explain why it does not express a clear bias towards tightening or easing monetary policy. Additionally, compare the sentence to a clear-cut example of hawkish and dovish language, and explain how the sente

Accuracy: 0.4444444444444444: 100%|██████████| 36/36 [00:47<00:00,  1.33s/it] 


...

Accuracy: 0.5277777777777778: 100%|██████████| 36/36 [00:53<00:00,  1.47s/it] 


Validation accuracy: 0.5277777777777778
Previous validation accuracy: 0.6388888888888888
Rejected prompt: Analyze the given sentence from FOMC and classify it as either hawkish, dovish, or neutral in terms of its stance on monetary policy tightening or easing. Provide a detailed explanation of your reasoning, highlighting the specific words and phrases that support your classification, and explaining why you believe the sentence leans towards a particular stance. If the sentence is neutral, please explain why it does not express a clear bias towards tightening or easing monetary policy. Additionally, provide a list of objective criteria that you used to make your classification, such as the use of words like 'optimistic' or 'cautious' to indicate a dovish stance, or the absence of words like 'aggressive' or 'tightening' to indicate a hawkish stance. Finally, provide a confidence score between 0 and 1, where 0 indicates no confidence and 1 indicates full confidence in your classificatio

Accuracy: 0.19444444444444445: 100%|██████████| 36/36 [00:49<00:00,  1.37s/it]


Validation accuracy: 0.19444444444444445
Previous validation accuracy: 0.6388888888888888
Rejected prompt: Analyze the given sentence from FOMC and classify its stance on monetary policy tightening or easing. Provide a detailed explanation of your reasoning, highlighting the specific words and phrases that support your classification, and explain why you believe the sentence leans towards a particular stance. If the sentence is neutral, please explain why it does not express a clear bias towards tightening or easing monetary policy. Additionally, provide a sentiment score between 0 and 1, where 0 indicates no confidence and 1 indicates full confidence in your classification.
sys prompt:  Analyze the given sentence from FOMC and classify it as either hawkish, dovish, or neutral in terms of its stance on monetary policy tightening or easing. Provide a detailed explanation of your reasoning, highlighting the specific words and phrases that support your classification, and explaining why y

Accuracy: 0.5833333333333334: 100%|██████████| 36/36 [00:00<00:00, 458.16it/s]
Training step 5. Epoch 4: : 5it [08:48, 105.66s/it]


In [40]:
pd.read_csv('optimization.csv')

,prompt,validation_accuracy,new_best
0,Classify the given sentence from FOMC as eithe...,0.472222,True
1,"For the purpose of this task, 'hawkish' refers...",0.444444,False
2,Classify the given sentence from FOMC as eithe...,0.444444,False
3,Classify the given sentence from FOMC as eithe...,0.527778,True
4,Classify the given sentence from FOMC as eithe...,0.527778,True
5,Classify the given sentence from FOMC as eithe...,0.472222,False
6,Analyze the given sentence from the FOMC meeti...,0.444444,False
7,Analyze the given sentence from FOMC and class...,0.555556,True
8,Analyze the given sentence from FOMC and expla...,0.222222,False
9,Analyze the given sentence from FOMC and provi...,0.000000,False


## Testing together 1.2 API

In [8]:
# Testing together 1.2 API
client = Together()
model_response = client.chat.completions.create(
    model='allenai/OLMo-7B-Instruct',
    messages=[{'role': 'system', 'content': fomc_prompt(sentence)[0]},
                {'role': 'user', 'content': fomc_prompt(sentence)[1]}],
    max_tokens=128,
    temperature=0.7,
    top_k=50,
    top_p=0.7,
    repetition_penalty=1.1
)
response_label = model_response.choices[0].message.content
token_usage = {'prompt_tokens': model_response.usage.prompt_tokens, 'response_tokens': model_response.usage.completion_tokens, 'total_tokens': model_response.usage.total_tokens}
print(f"LLM response: {response_label.strip()}\nToken usage: {token_usage}")

LLM response: Neutral class
Explanation: The sentence mentions that participants agreed that the labor market remained strong and economic activity rose at a moderate rate, indicating a neutral stance towards monetary policy.
Token usage: {'prompt_tokens': 174, 'response_tokens': 37, 'total_tokens': 211}


## Testing DSPy

In [54]:
# testing dspy
# dspy together module has some issues with how they wrote it (passing specific parameters is hard)
# rate limiting error messages
# TODO: look into dspy together module more to see how to fix it

lm = dspy.Together(model = 'mistralai/Mistral-7B-Instruct-v0.3', stop = ['</s>', '<s>'])
dspy.settings.configure(lm=lm)
d = {0: 'dovish', 1: 'hawkish', 2: 'neutral'}

class StanceAnalysis(dspy.Signature):
    """Classify the sentence's stance on the monetary policy between hawkish, neutral, and dovish."""
    
    sentence = dspy.InputField()
    stance = dspy.OutputField(desc = "hawkish, neutral, or dovish")

class Analysis(dspy.Module):
    def __init__(self):
        super().__init__()
        self.predict = dspy.Predict(StanceAnalysis)
    
    def forward(self, sentence):
        return self.predict(sentence=sentence)
    
analyze = Analysis()
analyze(sentence)

Backing off 0.6 seconds after 1 tries calling function <function Together._generate at 0x0000017878ABF700> with kwargs {'temperature': 0.0, 'max_tokens': 512, 'top_p': 1, 'top_k': 20, 'repetition_penalty': 1, 'n': 1, 'stop': ['</s>', '<s>']}
Backing off 0.5 seconds after 2 tries calling function <function Together._generate at 0x0000017878ABF700> with kwargs {'temperature': 0.0, 'max_tokens': 512, 'top_p': 1, 'top_k': 20, 'repetition_penalty': 1, 'n': 1, 'stop': ['</s>', '<s>']}
Backing off 3.4 seconds after 3 tries calling function <function Together._generate at 0x0000017878ABF700> with kwargs {'temperature': 0.0, 'max_tokens': 512, 'top_p': 1, 'top_k': 20, 'repetition_penalty': 1, 'n': 1, 'stop': ['</s>', '<s>']}
Backing off 7.0 seconds after 4 tries calling function <function Together._generate at 0x0000017878ABF700> with kwargs {'temperature': 0.0, 'max_tokens': 512, 'top_p': 1, 'top_k': 20, 'repetition_penalty': 1, 'n': 1, 'stop': ['</s>', '<s>']}
Backing off 12.9 seconds after 5

KeyboardInterrupt: 